
# models_analysis_enriched — Portable Version

Questo notebook calcola e unisce i conteggi di **Reason categories** (CAN, R, NR, CAR, AR, GP, BP, PR, AP)
dai file `redis_backup_db*.json` trovati nella sottocartella `results/`, con eventuale merge con `forest_report.csv`.

⚙️ Funziona in **qualsiasi percorso**: basta che nella stessa directory ci sia la cartella `results/`
e (facoltativamente) il file `forest_report.csv`.


In [33]:
from pathlib import Path
from drifts_results import compute_counts_from_results, load_analyzed_df, cast_dataset_str, CAT_LIST, DB_TO_CAT
# Detect notebook base directory as robustly as possible
try:
    from IPython import get_ipython
    ip = get_ipython()
    BASE_DIR = Path(ip.run_line_magic('pwd', '')).resolve()
except Exception:
    BASE_DIR = Path.cwd().resolve()
RESULTS_DIR = BASE_DIR / 'results'
FR_CSV = BASE_DIR / 'forest_report.csv'


SyntaxError: unmatched ')' (drifts_results.py, line 36)

In [ ]:

from pathlib import Path
import json, re, pandas as pd
from drifts_results import compute_counts_from_results, load_analyzed_df, cast_dataset_str, CAT_LIST, DB_TO_CAT

# === Rilevamento automatico dei percorsi (robusto per notebook) ===
try:
    from IPython import get_ipython
    ip = get_ipython()
    BASE_DIR = Path(ip.run_line_magic('pwd', '')).resolve()
except Exception:
    BASE_DIR = Path.cwd().resolve()
RESULTS_DIR = BASE_DIR / "results"
FR_CSV = BASE_DIR / "forest_report.csv"

print("📁 BASE_DIR     :", BASE_DIR)
print("📁 RESULTS_DIR  :", RESULTS_DIR)
print("📄 FOREST_REPORT:", FR_CSV.exists())


In [29]:


# === Calcolo e merge ===



counts_df = compute_counts_from_results(RESULTS_DIR, verbose=True)

analyzed_df = load_analyzed_df(FR_CSV)

counts_df   = cast_dataset_str(counts_df)

analyzed_df = cast_dataset_str(analyzed_df)



# Diagnostic: inspect zip archives for redis manifests
import zipfile
missing = []
for z in sorted(RESULTS_DIR.glob('*.zip')):
    try:
        with zipfile.ZipFile(z, 'r') as zz:
            names = zz.namelist()
            has = any('redis_backup_db' in n for n in names)
            if not has:
                missing.append((z.name, names[:10]))
    except Exception as e:
        missing.append((z.name, f'error: {e}'))

if missing:
    print('ZIP senza redis manifest (primi entry mostrati):')
    for name, sample in missing:
        print('-', name, '->', sample)
else:
    print('Tutti gli zip contengono redis_backup_db*.json (o non ci sono zip).')

# Restringi ai dataset presenti in results/

results_datasets = sorted(set([p.name.split("_")[0] if "_" in p.name else p.name

                               for p in RESULTS_DIR.glob("*") if p.is_dir() and any(p.rglob("redis_backup_db*.json"))]))

analyzed_df_res = analyzed_df[analyzed_df["dataset"].isin(results_datasets)].copy()



# Inner join

merged_results_only = analyzed_df_res.merge(counts_df, on="dataset", how="inner")



# Fill e cast

for c in DB_TO_CAT.values():

    if c not in merged_results_only.columns:

        merged_results_only[c] = 0

merged_results_only[list(DB_TO_CAT.values())] = merged_results_only[list(DB_TO_CAT.values())].fillna(0).astype(int)



# Aggiungi colonne derivate

merged_results_only["TOT"] = merged_results_only[list(DB_TO_CAT.values())].sum(axis=1)

merged_results_only["%R"]  = (100*merged_results_only["R"]/merged_results_only["TOT"]).round(2).replace([float("inf"),float("nan")],0)

merged_results_only["%NR"] = (100*merged_results_only["NR"]/merged_results_only["TOT"]).round(2).replace([float("inf"),float("nan")],0)



merged_results_only = merged_results_only.sort_values("R", ascending=False)



# Visualizza tabella

try:

    _ = style_summary_table

except NameError:

    def style_summary_table(df: pd.DataFrame):

        return (df.style.set_properties(**{"text-align":"left"})

                    .set_table_styles([{"selector":"th","props":[("text-align","left")]}]))

style_summary_table(merged_results_only)


,dataset,status,endpoints_universe,error,eu_complexity,mean_eu,n_features,test_score,validation_score,metadata_classes,metadata_dataset,metadata_first_class,metadata_length_consistent,metadata_n_channels,metadata_n_classes,metadata_series_length,metadata_test_size,metadata_train_size,best_params_bootstrap,best_params_ccp_alpha,best_params_criterion,best_params_max_depth,best_params_max_features,best_params_max_leaf_nodes,best_params_max_samples,best_params_min_impurity_decrease,best_params_min_samples_leaf,best_params_min_samples_split,best_params_n_estimators,best_params_random_state,forest_statistics_avg_depth,forest_statistics_avg_leaves,forest_statistics_avg_nodes,forest_statistics_max_depth,forest_statistics_min_depth,forest_statistics_n_estimators,analyzed,CAN,R,NR,CAR,AR,GP,BP,PR,AP,TOT,%R,%NR


In [30]:

# === Esporta CSV ===
OUT = BASE_DIR / "analyzed_counts_results_only.csv"
merged_results_only.to_csv(OUT, index=False)
print("✅ Esportato:", OUT)


✅ Esportato: /Users/danielamadori/Projects/drifts/analyzed_counts_results_only.csv
